In [2]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
IBMQ.load_accounts()

In [3]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua import Operator, run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import portfolio
from qiskit.aqua.translators.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, ExactEigensolver
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RY
import numpy as np
import datetime

In [27]:
#We are using a random portfolio problem.

#Number of assets (= number of qubits)
num_assets = 4

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
print (stocks)
#We use qiskit's RandomDataProvider fn to create ticker data-set
data = RandomDataProvider(tickers=stocks,
                 start=datetime.datetime(2018,6,1),
                 end=datetime.datetime(2018,6,30))
data.run()
print('Random Coordinates :')
print(data.get_coordinates())
print('Covariance Matrix :')
print (data.get_covariance_matrix())
print('Mean Vector :')
print (data.get_mean_vector())
print('A vector containing the mean value of each asset.')
mu = data.get_period_return_mean_vector()
print(mu)
print('Covariance Matrix :')
sigma = data.get_period_return_covariance_matrix()
print(sigma)

['TICKER0', 'TICKER1', 'TICKER2', 'TICKER3']
Random Coordinates :
(array([-0.37450115,  0.06161072,  0.48893157, -0.22045371]), array([ 0.02938386, -0.41893717,  0.05001787, -0.20037859]))
Covariance Matrix :
[[ 3.99360078 -0.71722479 -2.47469649 -2.50693025]
 [-0.71722479  2.10029804 -1.3202457  -0.21910134]
 [-2.47469649 -1.3202457   8.42485888  4.44438183]
 [-2.50693025 -0.21910134  4.44438183  4.14427693]]
Mean Vector :
[11.73352238 94.19713782 80.97469995 14.64087254]
A vector containing the mean value of each asset.
[ 0.0057441  -0.00194923  0.00273671  0.01274602]
Covariance Matrix :
[[ 1.12612659e-02  2.73288431e-04 -1.15646798e-04 -9.24185378e-04]
 [ 2.73288431e-04  1.00183399e-04  4.67567869e-05  3.63694815e-04]
 [-1.15646798e-04  4.67567869e-05  1.43826298e-04  2.82433270e-04]
 [-9.24185378e-04  3.63694815e-04  2.82433270e-04  7.52056293e-03]]


In [28]:
#Risk factor is q>0
q = 0.9 
#Budget, always less than the number of assets
budget = int(num_assets / 2) 
#parameter to scale the budget penalty term
penalty = num_assets 
#Using the above values and the mu and sigma
#We create an Operator instance for the Hamiltonian. 
#The Pauli operators are provided by an Ising Hamiltonian translated from the portfolio
qubitOp, offset = portfolio.get_portfolio_qubitops(mu, sigma, q, budget, penalty)
print(qubitOp)
print(qubitOp.print_operators(print_format='matrix')) #'paulis', 'matrix'
print ('Offset: '); print(offset)

#EnergyInput is a kind of AlgorithmInput - converts operators to a format suitable for algorithms
algo_input = EnergyInput(qubitOp)

Representation: paulis, qubits: 4, size: 10
[[ 1.20011103e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  5.50136190e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  3.14971535e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.99196732e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000

In [29]:
#Utility methods to print the results
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    #result - is a Dictionary
    selection = portfolio.sample_most_likely(result['eigvecs'][0])
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    probabilities = np.abs(result['eigvecs'][0])**2
    #np.argsort(probabilities) - returns the indices to sort asc 
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)    
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [30]:
#Lets solve the problem. First classically...
#Purpose is to have a classical reference. 

#ExactEigensolver, although from qiskit is a classical algorithm
#Backend is not required since this is computed classically not using quantum computation. 
#The params, along with the algo input containing the operator, 
#are now passed to the algorithm to be run. 
exact_eigensolver = ExactEigensolver(qubitOp, k=1)
result = exact_eigensolver.run()

""" the equivalent if using declarative approach
algorithm_cfg = {
    'name': 'ExactEigensolver'
}

params = {
    'problem': {'name': 'ising'},
    'algorithm': algorithm_cfg
}
result = run_algorithm(params, algo_input)
"""

#The result is returned as a dictionary.
print(result)
print_result(result)

{'eigvals': array([-4.00696609]), 'eigvecs': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]), 'energy': -4.006966085788148, 'wavefunction': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]), 'energies': array([-4.00696609])}
Optimal: selection [0 0 1 1], value -0.0081

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 0 1 1]	-0.0081		1.0000
 [1 1 1 1]	15.9977		0.0000
 [0 1 1 1]	3.9947		0.0000
 [1 0 1 1]	3.9944		0.0000
 [1 1 0 1]	3.9999		0.0000
 [0 1 0 1]	-0.0033		0.0000
 [1 0 0 1]	-0.0033		0.0000
 [0 0 0 1]	3.9940		0.0000
 [1 1 1 0]	4.0042		0.0000
 [0 1 1 0]	-0.0005		0.0000
 [1 0 1 0]	0.0016		0.0000
 [0 0 1 0]	3.9974		0.0000
 [1 1 0 0]	0.0069		0.0000
 [0 1 0 0]	4.0020		0.0000
 [1 0 0 0]	4.0044		0.0000
 [0 0 0 0]	16.0000		0.0000


In [31]:
#Solution using Quantum Algorithm Variational Quantum Eigensolver (VQE)
backend = BasicAer.get_backend('statevector_simulator')

seed = 50
#Specify the optimizer - Constrained Optimization BY Linear Approximation algorithm.
#It Uses scipy.optimize.minimize COBYLA  - this is classical
cobyla = COBYLA()
cobyla.set_options(maxiter=500)

#Specify the variational form - RY - Layers of Y rotations followed by entangling gates
#This is quantum-mechanical
ry = RY(qubitOp.num_qubits, depth=3, entanglement='full')

vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

"""declarative approach
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'COBYLA',
    'maxiter': 500
}

var_form_cfg = {
    'name': 'RY',
    'depth': 3,
    'entanglement': 'full'
}

params = {
    'problem': {'name': 'ising', 'random_seed': seed},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
result = run_algorithm(params, algo_input, backend=backend)
"""
print_result(result)

Optimal: selection [1 1 0 0], value 0.0069

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [1 1 0 0]	0.0069		0.6893
 [1 0 1 0]	0.0016		0.2997
 [0 0 1 1]	-0.0081		0.0098
 [0 1 0 1]	-0.0033		0.0008
 [1 0 0 1]	-0.0033		0.0003
 [1 0 1 1]	3.9944		0.0000
 [1 1 0 1]	3.9999		0.0000
 [0 1 1 0]	-0.0005		0.0000
 [0 1 0 0]	4.0020		0.0000
 [0 0 1 0]	3.9974		0.0000
 [1 0 0 0]	4.0044		0.0000
 [0 0 0 0]	16.0000		0.0000
 [0 0 0 1]	3.9940		0.0000
 [0 1 1 1]	3.9947		0.0000
 [1 1 1 1]	15.9977		0.0000
 [1 1 1 0]	4.0042		0.0000


In [32]:
#Solution using Quantum Approximate Optimization Algorithm (QAOA)
#Another variational algorithm that uses an internal variational form, created based on the problem.
backend = BasicAer.get_backend('statevector_simulator')
seed = 50

#Specify the optimizer - Constrained Optimization BY Linear Approximation algorithm.
#It Uses scipy.optimize.minimize COBYLA  
cobyla = COBYLA()
cobyla.set_options(maxiter=250)

qaoa = QAOA(qubitOp, cobyla, 3)
qaoa.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_transpiler=seed)

result = qaoa.run(quantum_instance)

"""declarative approach
algorithm_cfg = {
    'name': 'QAOA.Variational',
    'p': 3,
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'COBYLA',
    'maxiter': 250
}

params = {
    'problem': {'name': 'ising', 'random_seed': seed},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg
}
result = run_algorithm(params, algo_input, backend=backend)
"""
print_result(result)

Optimal: selection [0 0 1 1], value -0.0081

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 0 1 1]	-0.0081		0.1668
 [1 0 0 1]	-0.0033		0.1667
 [0 1 0 1]	-0.0033		0.1667
 [0 1 1 0]	-0.0005		0.1667
 [1 0 1 0]	0.0016		0.1666
 [1 1 0 0]	0.0069		0.1665
 [1 0 1 1]	3.9944		0.0000
 [0 1 1 1]	3.9947		0.0000
 [1 1 0 1]	3.9999		0.0000
 [0 0 0 1]	3.9940		0.0000
 [0 0 1 0]	3.9974		0.0000
 [1 1 1 0]	4.0042		0.0000
 [0 1 0 0]	4.0020		0.0000
 [1 0 0 0]	4.0044		0.0000
 [1 1 1 1]	15.9977		0.0000
 [0 0 0 0]	16.0000		0.0000
